In [6]:
from decouple import config
import tweepy
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

In [7]:
# Get database credentials, db path and twitter bearer token from env-file
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = config('PORT')
dbname = config('DBNAME')
bearer_token = config("BEARER_TOKEN")

In [9]:
# Define API client function
client = tweepy.Client(bearer_token=bearer_token)

In [13]:
def fetch_load_tweetcount(label, ht1, ht2, start, end, method):
    # Define query, time period and table_name
    query = '(#bitcoin OR #btc) -is:retweet'
    query = '('+ ht1 + ' OR ' + ht2 + ') -is:retweet'
    print(query)
    start = start
    end = end
    table_name = 'table_tw'+label
    print(table_name)
        
    while True:    
        #fetch data
        try:
            counts = client.get_recent_tweets_count(query=query,start_time = start, end_time = end , granularity='day')
            tweets_df = pd.DataFrame()

            for count in counts.data:
                tweets_df = tweets_df.append(pd.DataFrame([{'date': count['end'], 
                                                'tweet_count': count['tweet_count']}]))

            tweets_df['date'] = pd.to_datetime(tweets_df['date'])
            tweets_df['label'] = label
            tweets_df = tweets_df.reset_index(drop=True)                    
        except Error as e:
            print(e)
            print("tweet count failed: " + label)
            break
        
                
        #load data
        if_ex_val = method
        
        try:
            conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host1+':'+port+'/'+dbname
            engine = create_engine(conn_string)
            tweets_df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
        except Exception as e:
            print(e)
            print("Data load failed: " + table_name)   
            break
        break   

In [11]:
hashtags = {'bitcoin': ['#bitcoin','#btc'],'ethereum': ['#eth', '#ethereum'], 'binance': ['#binance','#bnb'],'ripple': ['#ripple','#xrp'],
           'terra':['#terra','#luna'], 'cardano': ['#cardano','#ada'], 'solana': ['#sol','#solana'], 'avalanche': ['#avalanche','#avax'],
           'polkadot':['#polkadot','#dot'],'dogecoin':['#dogecoin', '#doge'], 'msci_world': ['#msciworld', '#msci'],'euro_stoxx': ['#EXW1.DE','#eurostoxx'],
            'smi':['#CSSMI.SW', '#smi'], 'nasdaq': ['#nasdaq', '#EXXT.DE'], 'gold':['#gold','#GC=F'], 'silver':['#silver','#SI=F']}

In [ ]:
# Initial load of the historical data
yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
firstday = datetime.strftime(datetime.today() - timedelta(6), "%Y, %m, %d")

# set timeframe
end = datetime.strptime(yesterday, "%Y, %m, %d")
start = datetime.strptime(firstday, "%Y, %m, %d")


# set method of function df.to_sql()
method = 'replace'

# execute function
for label in hashtags:
    ht1 = hashtags[label][0]
    ht2 = hashtags[label][1]
    fetch_load_tweetcount(label, ht1, ht2, start, end, method)

(#bitcoin OR #btc) -is:retweet
table_bitcoin
(#eth OR #ethereum) -is:retweet
table_ethereum
(#binance OR #bnb) -is:retweet
table_binance
(#ripple OR #xrp) -is:retweet
table_ripple
(#terra OR #luna) -is:retweet
table_terra
(#cardano OR #ada) -is:retweet
table_cardano
(#sol OR #solana) -is:retweet
table_solana
(#avalanche OR #avax) -is:retweet
table_avalanche
(#polkadot OR #dot) -is:retweet
table_polkadot
(#dogecoin OR #doge) -is:retweet
table_dogecoin
(#msciworld OR #msci) -is:retweet
table_msci_world
(#EXW1.DE OR #eurostoxx) -is:retweet
table_euro_stoxx
(#CSSMI.SW OR #smi) -is:retweet
table_smi
(#nasdaq OR #EXXT.DE) -is:retweet
table_nasdaq
(#gold OR #GC=F) -is:retweet
table_gold
